## Alphalens の勉強

https://www.quantopian.com/posts/alphalens-a-new-tool-for-analyzing-alpha-factors


In [ ]:
import numpy as np
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import CustomFactor, Returns, AverageDollarVolume, 
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.filters import Q500US
MORNINGSTAR_SECTOR_CODES = {
     -1: 'Misc',
    101: 'Basic Materials',
    102: 'Consumer Cyclical',
    103: 'Financial Services',
    104: 'Real Estate',
    205: 'Consumer Defensive',
    206: 'Healthcare',
    207: 'Utilities',
    308: 'Communication Services',
    309: 'Energy',
    310: 'Industrials',
    311: 'Technology' ,    
}


### 自分のCustom Factorを定義

In [ ]:
class LowVol(CustomFactor):
    inputs = [Returns(window_length=2)] # 前日比
    window_length=25
    
    def compute(self, today, assets, out, close):
        out[:] = -np.nanstd(close, axis=0) # NaN を無視してStdを計算


### 自分のUniverseを定義

In [ ]:
universe = Q500US()

### 自分のPipelineを作成

In [ ]:
pipe_low_vol = Pipeline()
pipe_low_vol.add(LowVol(mask=universe), "LowVol")
pipe_low_vol.add( Sector(mask=universe), "Sector")
pipe_low_vol.set_screen(universe)


### Pipeline 実行

In [ ]:
results = run_pipeline(pipe_low_vol, '2015-6-30', '2016-6-30')

### 価格データを取得
+ Pipelineよりも少し長いデータを取得して，将来のReturn も結果に反映できるように，別件で価格を取得

In [ ]:
assets = results.index.levels[1].unique()
pricing = get_pricing(assets, start_date='2015-6-30', end_date='2016-7-30', fields='open_price')

### Alphalens tear sheet を作ろう

In [ ]:
import alphalens
factor_data = alphalens.utils.get_clean_factor_and_forward_returns(
    results['LowVol'],
    pricing, 
    quantiles=5,
    groupby=results['Sector'],
    periods=(1,5,10))

 ##[get_clean_factor_and_forward_returns](http://quantopian.github.io/alphalens/alphalens.html?highlight=get_clean_factor_and_forward_returns#alphalens.utils.get_clean_factor_and_forward_returns)
 
**パラメタ**
+ **factor**, Multiindex (date, asset) の**1ファクター**データ．
+ **prices**, date をindex， assets を columns に持つDataFrame．**ルックアヘッドバイアスを防ぐ為に，ファクターが作られたTimePeriodと同じでなくてはならず，また，Factorよりも長めのデータを渡す必要がある．**
+ **groupby**=None, Multiindex (date, asset) の1ファクターデータ，もしくは，辞書．辞書の場合は，Keｙにasset
+ **by_group**=False, Trueの場合は，グループ毎に統計演算する．
+ **quantiles**=5, Intもしくは，Floatのリスト．intの場合は，その数でファクターを等分位.リスト（例；[0, .10, .5, .90, 1.] or [.05, .5, .95]）は，その区切りで分ける．わけた分位毎に結果を出す　**quantile もしくは bin のどちらかのみNone**
+ **bins**=None, Intもしくは，Floatのリスト．intの場合はbin数．リスト（例； [-4, -2, -0.5, 0, 10]）の場合はそれでBinをつくる．
    + 注意！**bins=**を指定するときは必ず **quantiles=None** を明示しないと，例外が発生する
+ **periods**=(1, 5, 10), intのリスト．何日後のリターンを見るか．（デフォルトだと，翌日，5日後，10日後）
+ **filter_zscore**=20, int もしくは float. リターンをz値でフィルターする．このハズレ値フィルタリングは，ルックアヘッドバイアスしてしまう可能性がある．
+ **groupby_labels**=None．それぞれのグループに別名を付けたいときつかう. None の場合は ”factor” になる

**返り値**

Multiindex (date, asset) ，のDataFrameで，パラメータで渡した結果が，コラムに入っているデータ

> factor_data.head()
 
date|asset|1|5|10|factor|group|factor_quantile
---|---|---|---|---|---|---|---
2015-06-30 00:00:00+00:00|Equity(2 [ARNC])|-0.021778|-0.043644|-0.059313|-0.008829|101|5
|Equity(24 [AAPL])|0.010597|-0.008679|0.001195|-0.008808|311|5
|Equity(62 [ABT])|-0.002225|0.004637|0.01152|-0.009697|206|4
|Equity(67 [ADSK])|-0.009787|0.031904|0.0413|-0.013364|311|2
|Equity(76 [TAP])|-0.01249|-0.013334|0.007296|-0.012557|205|2




 ##[create_full_tear_sheet](http://quantopian.github.io/alphalens/alphalens.html?highlight=create_full_tear_sheet#alphalens.tears.create_full_tear_sheet)
 
1ファクターを評価するためのティアシートを作成

**パラメタ**

+ **factor_data**：MultiindexのDataFrame．get_clean_factor_and_forward_returnsの返り値のようなデータ．
+ **long_short**=True, Long Short両方あるストラテジーか？
+ **group_adjust**=False, ICを計算する前に，グループ毎のリターンを個別に戻すか？
+ **by_group**=False　グループごとに結果をだすか？


In [ ]:
alphalens.tears.create_full_tear_sheet(factor_data, by_group=True)

# よくあるエラー集

エラー：<font color=red>ValueError: Bin edges must be unique: array([  0.,   0.,   0.,   0.,   0.,  10.])</font>

解決： bins = を指定して， quantiles=None, を明示する． quantiles=None, はデフォルトなんだけど，binを指定するときは明示しないと何故か例外を返す


参照： [No objects to concatenate，using alaphalens to analysing my own factor](https://www.quantopian.com/posts/no-objects-to-concatenate-using-alaphalens-to-analysing-my-own-factor#59a6ba1c5b2059000dfa91b3)
